# APS360 Group Project

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from train import train_net, train_auto_encoder, init_device, test_net
from graph import plot_training_curve, generate_confusion_matrix, visualize_output, visualize_autoencoder_output

In [2]:
init_device()

Cuda Available: False


## Baseline Model

In [4]:
class BaselineModel(nn.Module):
    def __init__(self):
        super(BaselineModel, self).__init__() 
        self.conv1 = nn.Conv2d(1, 5, 3) 
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(5, 10, 3)
        self.conv3 = nn.Conv2d(10, 20, 3)
        self.fc = nn.Linear(26*26*20, 10)

    def forward(self, x, training=False):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 26*26*20)

        x = self.fc(x)

        return x


In [12]:
train_net(BaselineModel, "baseline_model", learning_rate=0.01, num_epochs=15)

Done Loading Data
Starting Training
Epoch 1: Train err: 0.7708333333333334, Train loss: 2.1306509159980935 | Validation err: 0.607, Validation loss: 1.6415250524878502
Epoch 2: Train err: 0.5245, Train loss: 1.3954660283758284 | Validation err: 0.3895, Validation loss: 1.1280820220708847
Epoch 3: Train err: 0.39116666666666666, Train loss: 1.0884576059402304 | Validation err: 0.384, Validation loss: 1.0288156121969223
Epoch 4: Train err: 0.345, Train loss: 0.8720334349794591 | Validation err: 0.349, Validation loss: 0.8103780001401901
Epoch 5: Train err: 0.31116666666666665, Train loss: 0.7291760812414453 | Validation err: 0.336, Validation loss: 0.7149652540683746
Epoch 6: Train err: 0.24583333333333332, Train loss: 0.5668112121997996 | Validation err: 0.2435, Validation loss: 0.545511532574892
Epoch 7: Train err: 0.195, Train loss: 0.44154488152646004 | Validation err: 0.224, Validation loss: 0.48631178960204124
Epoch 8: Train err: 0.37283333333333335, Train loss: 1.146186482399068 |

## Primary Model

In [5]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1, 1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, 1, 1)
        self.conv3 = nn.Conv2d(64, 128, 3, 1, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))    # 32 x 112 x 112
        x = self.pool(F.relu(self.conv2(x)))    # 64 x 56 x 56
        x = self.pool(F.relu(self.conv3(x)))    # 128 x 28 x 28

        return x

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()

        self.conv4 = nn.ConvTranspose2d(128, 64, 3, 2, 1, 1)
        self.conv5 = nn.ConvTranspose2d(64, 32, 3, 2, 1, 1)
        self.conv6 = nn.ConvTranspose2d(32, 1, 3, 2, 1, 1)

    def forward(self, x):

        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x)) 
        x = F.relu(self.conv6(x))
        
        return x

class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)

        return x

class PrimaryModel(nn.Module):
    def __init__(self, encoder):
        super(PrimaryModel, self).__init__()
        self.encoder = encoder
        self.encoder.requires_grad_(False)

        self.conv = nn.Conv2d(128, 32, 1)
        
        self.fc1 = nn.Linear(32*28*28, 1024)
        self.fc2 = nn.Linear(1024, 10)

    def forward(self, x):
        
        x = self.encoder(x)
        x = F.relu(self.conv(x))
        
        x = x.view(-1, 32*28*28)
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x), dim=1)
        
        return x

In [14]:
train_auto_encoder(AutoEncoder, "autoencoder", learning_rate=0.005, num_epochs=30)

Done Loading Data
Starting Training
Epoch 1: Train loss: 0.35112143212810476 | Validation loss: 0.0658773377072066
Epoch 2: Train loss: 0.05716663353303646 | Validation loss: 0.05083732237108052
Epoch 3: Train loss: 0.04851811442603456 | Validation loss: 0.04680552030913532
Epoch 4: Train loss: 0.043679443683396 | Validation loss: 0.041684751864522696
Epoch 5: Train loss: 0.04050125577982436 | Validation loss: 0.039891503751277924
Epoch 6: Train loss: 0.03919015388856543 | Validation loss: 0.038856480503454804
Epoch 7: Train loss: 0.03837476710372783 | Validation loss: 0.03822302143089473
Epoch 8: Train loss: 0.03789618944234036 | Validation loss: 0.037732705706730485
Epoch 9: Train loss: 0.03748569922878387 | Validation loss: 0.0374513054266572
Epoch 10: Train loss: 0.03715562344865596 | Validation loss: 0.03706261143088341
Epoch 11: Train loss: 0.036836005588795274 | Validation loss: 0.03681260207667947
Epoch 12: Train loss: 0.036572128851362996 | Validation loss: 0.03663111873902380

KeyboardInterrupt: 

In [7]:

auto_encoder = AutoEncoder()
auto_encoder.load_state_dict(torch.load("autoencoder\\best_model"))
train_net(PrimaryModel, "primary_model", encoder=auto_encoder.encoder, learning_rate=0.001, num_epochs=30)

Done Loading Data
Starting Training


# Visualization

In [ ]:
# Testing
print("Baseline Model: ", end="")
test_net(BaselineModel, "baseline_model\\best_model")
print("Primary Model: ", end="")
test_net(PrimaryModel, "primary_model\\best_model")

In [ ]:
# Graphing
plot_training_curve("baseline_model")
plot_training_curve("primary_model")

In [ ]:
# Visualizing
visualize_output(5 ,"baseline_model\\best_model", BaselineModel)
visualize_autoencoder_output(5, "autoencoder\\best_model", AutoEncoder)
visualize_output(5, "primary_model\\best_model", PrimaryModel)

In [ ]:
# Confusion Matrix
generate_confusion_matrix(BaselineModel, "baseline_model\\best_model")
generate_confusion_matrix(PrimaryModel, "primary_model\\best_model")